In [6]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report,roc_auc_score,accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from scikeras.wrappers import KerasClassifier

In [7]:
df=pd.read_csv('processed_outliers.csv')
print('Data shape:',df.shape)

Data shape: (1159, 22)


In [8]:
list(enumerate(df.columns))

[(0, 'w/b'),
 (1, 'Water'),
 (2, 'Cement type'),
 (3, 'Cement'),
 (4, 'Slag'),
 (5, 'Fly ash'),
 (6, 'Silica fume'),
 (7, 'Lime filler'),
 (8, 'FA'),
 (9, 'CA'),
 (10, 'Plasticizer'),
 (11, 'Superplasticizer'),
 (12, 'Air entraining'),
 (13, 'Comp. str. test age'),
 (14, 'Compressive strength'),
 (15, 'Air content'),
 (16, 'Spreed'),
 (17, 'Slump'),
 (18, 'Fresh density'),
 (19, 'Dry  density'),
 (20, 'Migration test age'),
 (21, 'Migration resistance')]

1-hot encoding

In [9]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [2])], remainder='passthrough')
X=ct.fit_transform(df)
# remove dummy variable and output
y=X[:,-1]
X=X[:,1:-1]
print(X.shape)
print(y.shape)

(1159, 30)
(1159,)


ANN

In [10]:
def get_ann(optimizer='rmsprop', init='glorot_uniform'):
    ann=keras.models.Sequential()
    ann.add(keras.layers.Dense(60,activation='relu',kernel_initializer=init,
                            input_shape=(X.shape[1],)))
    ann.add(keras.layers.Dense(32,activation='relu'))
    ann.add(keras.layers.Dense(32,activation='relu'))
    ann.add(keras.layers.Dense(5,activation='softmax'))
    ann.compile(optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return ann

In [11]:
param_grid={
    'optimizer': ['rmsprop', 'adam'],
    'model__init': ['glorot_uniform', 'normal', 'uniform'],
    'epochs': [200,400],
    'batch_size': [16,32,64]
}

splits=list(StratifiedKFold(shuffle=True,random_state=0).split(X,y))
train_index, test_index = splits[0]
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

def train_ann(X_train,y_train):
    grid=GridSearchCV(KerasClassifier(get_ann,random_state=0,verbose=0),
                      param_grid,cv=5,n_jobs=3,scoring='accuracy')
    grid.fit(X_train, y_train)
    print(grid.best_params_)
    print(grid.best_score_)
    return grid

In [14]:
grid=train_ann(X_train,y_train)

d:\College files\9th sem\mtp\code\.venv\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


{'batch_size': 16, 'epochs': 200, 'model__init': 'uniform', 'optimizer': 'rmsprop'}
0.8092182505085731


In [15]:
best_model=KerasClassifier(get_ann,random_state=0,verbose=0,**grid.best_params_)
best_model.fit(X_train,y_train)
print(classification_report(y_test,best_model.predict(X_test)))
roc_auc_score(y_test,best_model.predict_proba(X_test),multi_class='ovr')

              precision    recall  f1-score   support

         0.0       0.86      0.83      0.84        46
         1.0       0.84      0.81      0.83        47
         2.0       0.77      0.87      0.82        46
         3.0       0.95      0.87      0.91        47
         4.0       0.96      1.00      0.98        46

    accuracy                           0.88       232
   macro avg       0.88      0.88      0.88       232
weighted avg       0.88      0.88      0.88       232



0.9735659247024172

Removing cement type

In [12]:
X=df.drop('Cement type',axis=1).iloc[:,:-1].values
y=df.iloc[:,-1].values
print(X.shape)
print(y.shape)

(1159, 20)
(1159,)


In [13]:
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [18]:
grid=train_ann(X_train,y_train)

d:\College files\9th sem\mtp\code\.venv\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


{'batch_size': 32, 'epochs': 200, 'model__init': 'uniform', 'optimizer': 'rmsprop'}
0.8415228131357164


In [14]:
best_params={'batch_size': 32, 'epochs': 200, 'model__init': 'uniform', 'optimizer': 'rmsprop'}
best_model=KerasClassifier(get_ann,random_state=0,verbose=0,**best_params)
best_model.fit(X_train,y_train)
print(classification_report(y_test,best_model.predict(X_test)))
roc_auc_score(y_test,best_model.predict_proba(X_test),multi_class='ovr')

              precision    recall  f1-score   support

           0       0.93      0.89      0.91        46
           1       0.90      0.77      0.83        47
           2       0.80      0.87      0.83        46
           3       0.87      0.98      0.92        47
           4       0.98      0.96      0.97        46

    accuracy                           0.89       232
   macro avg       0.90      0.89      0.89       232
weighted avg       0.90      0.89      0.89       232



0.9752074349124571

In [15]:
accuracy_score(y_test,best_model.predict(X_test))

0.8922413793103449